# Logging

## Intuition

Logging is the process of tracking and recording key events that occur in our applications. We want to log events so we can use them to inspect processes, fix issues, etc. They're a whole lot more powerful than print statements because they allow us to send specific pieces of information to specific locations, not to mention custom formatting, shared interface with other Python packages, etc. This makes logging a key proponent in being able to surface insightful information from the internal processes of our application.

## Components

There are a few overarching concepts to be aware of first before we can create and use our loggers.

- `Logger`: the main object that emits the log messages from our application.
- `Handler`: used for sending log records to a specific location and specifications for that location (name, size, etc.).
- `Formatter`: used for style and layout of the log records.

There is so much [more](https://docs.python.org/3/library/logging.html) to logging such as filters, exception logging, etc. but these basics will allows us to do everything we need for our application.

## Logging Levels

The numeric values of logging levels are given in the following table. These are primarily of interest if you want to define your own levels, and need them to have specific values relative to the predefined levels. If you define a level with the same numeric value, it overwrites the predefined value; the predefined name is lost.

| Level | Value |
| ----- | ----- |
| DEBUG | 10    |
| INFO  | 20    |
| WARN  | 30    |
| ERROR | 40    |
| FATAL | 50    |

Before we create our specialized, configured logger, let's look at what logged messages even look like by using a very basic configuration.

In [ ]:
import logging
import sys
from typing import Optional

These are the basic levels of logging, where `DEBUG` is the lowest priority and `CRITICAL` is the highest. We defined our logger using `basicConfig` to emit log messages to our `stdout` console (we also could've written to any other stream or even a file) and to be sensitive to log messages starting from level `DEBUG`. This means that all of our logged messages will be displayed since `DEBUG` is the lowest level. Had we made the level `ERROR`, then only `ERROR` and `CRITICAL` log message would be displayed. (madewithml.com)


In our first example, we set the `level` to be `logging.DEBUG` and all $5$ messages are logged, as can be seen.

In [ ]:
# Create super basic logger
logging.basicConfig(stream=sys.stdout, level=logging.DEBUG)

# Logging levels (from lowest to highest priority)
logging.debug("Used for debugging your code.")
logging.info("Informative messages from your code.")
logging.warning("Everything works but there is something to be aware of.")
logging.error("There's been a mistake with the process.")
logging.critical("There is something terribly wrong and process may terminate.")

ERROR:root:There's been a mistake with the process.
CRITICAL:root:There is something terribly wrong and process may terminate.


In the next example, we set the `level` to be `logging.ERROR`, this means all messages lower than `error` is ignored, as can be seen here! Note, if you are working in google colab, one should factory reset the notebook so that the `logger` can be refreshed.

In [ ]:
# Create super basic logger
logging.basicConfig(stream=sys.stdout, level=logging.ERROR)

# Logging levels (from lowest to highest priority)
logging.debug("Used for debugging your code.")
logging.info("Informative messages from your code.")
logging.warning("Everything works but there is something to be aware of.")
logging.error("There's been a mistake with the process.")
logging.critical("There is something terribly wrong and process may terminate.")

ERROR:root:There's been a mistake with the process.
CRITICAL:root:There is something terribly wrong and process may terminate.


## Custom Logger Function

We will define a custom logger function for our purpose. 

> If you encounter `logger` printing the same line multiple times, we should factory reset runtime.

In [ ]:
import logging
import sys

from pathlib import Path
from typing import Optional

We created a logging directory in the section **Organization**, however, for clarity, we create the `LOGS_DIR` again below (won't be overwritten). We will send all our logs to this directory.

In [ ]:
# Creating Directories
BASE_DIR = Path("__file__").parent.absolute()
LOGS_DIR = Path(BASE_DIR, "logs")
LOGS_DIR.mkdir(parents=True, exist_ok=True)

In [ ]:
# Logger
def init_logger(
    logs_dir: Path,
    default_level=logging.DEBUG,
    stream_level=logging.INFO,
    module_name: Optional[str] = None,
) -> logging.Logger:
    """Initialize logger.

    Args:
        logs_dir (Path): Path to log directory.
        default_level (int, optional): Default logging level. Defaults to logging.DEBUG.
        stream_level (int, optional): Stream logging level. Defaults to logging.INFO.
        module_name (Optional[str]): Module name to be used in logger. Defaults to None.

    Returns:
        logging.Logger: The logger object.

    Example:
        >>> import logging
        >>> import sys
        >>> from pathlib import Path
        >>> from typing import Optional
        >>> # Creating Directories
        >>> BASE_DIR = Path("__file__").parent.parent.absolute()
        >>> LOGS_DIR = Path(BASE_DIR, "logs")
        >>> LOGS_DIR.mkdir(parents=True, exist_ok=True)
        >>> train_logger = init_logger(LOGS_DIR, module_name="train")
        >>> # Logging levels (from lowest to highest priority)
        >>> try:
        >>>     train_logger.info("I am trying to divide by zero!")
        >>>     1 / 0
        >>> except ZeroDivisionError as e:
        >>>     train_logger.error(e)  # ERROR:root:division by zero
        >>>     train_logger.critical(e, exc_info=True)  # Logs error with stack trace
    """

    if module_name is None:
        logger = logging.getLogger(__name__)
        info_log_filepath = Path(logs_dir, "info.log")
        error_log_filepath = Path(logs_dir, "error.log")
    else:
        # get module name, useful for multi-module logging
        logger = logging.getLogger(module_name)
        info_log_filepath = Path(logs_dir, f"{module_name}_info.log")
        error_log_filepath = Path(logs_dir, f"{module_name}_error.log")

    logger.setLevel(default_level)
    stream_handler = logging.StreamHandler(stream=sys.stdout)
    stream_handler.setLevel(stream_level)
    stream_handler.setFormatter(
        logging.Formatter(
            "%(asctime)s - %(name)s - %(levelname)s: %(message)s",
            "%Y-%m-%d %H:%M:%S",
        )
    )

    info_file_handler = logging.FileHandler(filename=info_log_filepath)
    info_file_handler.setLevel(logging.INFO)
    info_file_handler.setFormatter(
        logging.Formatter(
            "%(asctime)s - %(name)s - %(levelname)s: %(message)s",
            "%Y-%m-%d %H:%M:%S",
        )
    )

    # add error file handler
    error_file_handler = logging.FileHandler(filename=error_log_filepath)
    error_file_handler.setLevel(logging.ERROR)
    error_file_handler.setFormatter(
        logging.Formatter(
            "%(asctime)s - %(name)s - %(levelname)s: %(message)s",
            "%Y-%m-%d %H:%M:%S",
        )
    )

    logger.addHandler(stream_handler)
    logger.addHandler(info_file_handler)
    logger.addHandler(error_file_handler)
    logger.propagate = False
    return logger

In [ ]:
train_logger = init_logger(
    logs_dir = LOGS_DIR,
    default_level = logging.DEBUG,
    stream_level=logging.INFO,
    module_name = "train")

In [ ]:
# Logging levels (from lowest to highest priority)
train_logger.debug("Used for debugging your code.")
train_logger.info("Informative messages from your code.")
train_logger.warning("Everything works but there is something to be aware of.")
train_logger.error("There's been a mistake with the process.")
train_logger.critical("There is something terribly wrong and process may terminate.")

2022-04-27 09:55:29 - train - INFO: Informative messages from your code.
2022-04-27 09:55:29 - train - WARNING: Everything works but there is something to be aware of.
2022-04-27 09:55:29 - train - ERROR: There's been a mistake with the process.
2022-04-27 09:55:29 - train - CRITICAL: There is something terribly wrong and process may terminate.


Lo and behold, the `train_logger` is behaving properly:
- console level: all messages above `INFO` are printed.
- info file: all messages above `INFO` are logged in the file.
- error file: all messages above `DEBUG` are logged in the file, in particular, messages of lower priority like `.info` and `.debug` are not logged.

The reason of having $2$ log files is that one file (info) logs almost everything, while the other (error) only logs the error messages etc. This avoids clutter and eases developer to pin-point errors when reviewing the code.

For completeness sake, we define another `logger` called `inference_logger` and see that it behaves the same, except for the fact that it is logging messages for another module.

In [ ]:
inference_logger = init_logger(
    logs_dir = LOGS_DIR,
    default_level = logging.DEBUG,
    stream_level=logging.INFO,
    module_name = "inference")

In [ ]:
# Logging levels (from lowest to highest priority)
inference_logger.debug("Used for debugging your code.")
inference_logger.info("Informative messages from your code.")
inference_logger.warning("Everything works but there is something to be aware of.")
inference_logger.error("There's been a mistake with the process.")
inference_logger.critical("There is something terribly wrong and process may terminate.")

2022-04-27 09:55:33 - inference - INFO: Informative messages from your code.
2022-04-27 09:55:33 - inference - WARNING: Everything works but there is something to be aware of.
2022-04-27 09:55:33 - inference - ERROR: There's been a mistake with the process.
2022-04-27 09:55:33 - inference - CRITICAL: There is something terribly wrong and process may terminate.


## Example Usage

The below small example shows how one can log messages. In particular, in the `except` clause, we called `logging.error(e)` to log the error messages and `logging.critical(e, exc_info=True)` to log both the message and the stack trace.

In [ ]:
%%writefile {CONFIG_DIR}/config.py
import logging
import sys
from dataclasses import dataclass
from pathlib import Path
from typing import List, Optional
import datetime

# Creating Directories
BASE_DIR = Path(__file__).parent.parent.absolute()

CONFIG_DIR = Path(BASE_DIR, "config")
LOGS_DIR = Path(BASE_DIR, "logs")
SRC_DIR = Path(BASE_DIR, "src")
DATA_DIR = Path(BASE_DIR, "data")
STORES_DIR = Path(BASE_DIR, "stores")
TEST_DIR = Path(BASE_DIR, "tests")

## Local stores
MODEL_REGISTRY = Path(STORES_DIR, "model")
RAW_DATA = Path(DATA_DIR, "raw")
PROCESSED_DATA = Path(DATA_DIR, "processed")

## Create dirs
for d in [
    CONFIG_DIR,
    LOGS_DIR,
    DATA_DIR,
    STORES_DIR,
    TEST_DIR,
    MODEL_REGISTRY,
    RAW_DATA,
    PROCESSED_DATA,
]:
    d.mkdir(parents=True, exist_ok=True)


# Logger
def init_logger(
    logs_dir: Path,
    default_level=logging.DEBUG,
    stream_level=logging.INFO,
    module_name: Optional[str] = None,
) -> logging.Logger:
    """Initialize logger.

    Args:
        logs_dir (Path): Path to log directory.
        default_level (int, optional): Default logging level. Defaults to logging.DEBUG.
        stream_level (int, optional): Stream logging level. Defaults to logging.INFO.
        module_name (Optional[str]): Module name to be used in logger. Defaults to None.

    Returns:
        logging.Logger: The logger object.

    Example:
        >>> import logging
        >>> import sys
        >>> from pathlib import Path
        >>> from typing import Optional
        >>> # Creating Directories
        >>> BASE_DIR = Path("__file__").parent.parent.absolute()
        >>> LOGS_DIR = Path(BASE_DIR, "logs")
        >>> LOGS_DIR.mkdir(parents=True, exist_ok=True)
        >>> train_logger = init_logger(LOGS_DIR, module_name="train")
        >>> # Logging levels (from lowest to highest priority)
        >>> try:
        >>>     train_logger.info("I am trying to divide by zero!")
        >>>     1 / 0
        >>> except ZeroDivisionError as e:
        >>>     train_logger.error(e)  # ERROR:root:division by zero
        >>>     train_logger.critical(e, exc_info=True)  # Logs error with stack trace
    """

    datetime_ = datetime.datetime.now().strftime("%Y-%m-%d_%H-%M-%S")
    if module_name is None:
        logger = logging.getLogger(__name__)
        info_log_filepath = Path(logs_dir, f"{datetime_}_info.log")
        error_log_filepath = Path(logs_dir, f"{datetime_}_error.log")
    else:
        # get module name, useful for multi-module logging
        logger = logging.getLogger(module_name)
        info_log_filepath = Path(
            logs_dir, f"{datetime_}_{module_name}_info.log"
        )
        error_log_filepath = Path(
            logs_dir, f"{datetime_}_{module_name}_error.log"
        )

    logger.setLevel(default_level)
    stream_handler = logging.StreamHandler(stream=sys.stdout)
    stream_handler.setLevel(stream_level)
    stream_handler.setFormatter(
        logging.Formatter(
            "%(asctime)s - %(name)s - %(levelname)s: %(message)s",
            "%Y-%m-%d %H:%M:%S",
        )
    )

    info_file_handler = logging.FileHandler(filename=info_log_filepath)
    info_file_handler.setLevel(logging.INFO)
    info_file_handler.setFormatter(
        logging.Formatter(
            "%(asctime)s - %(name)s - %(levelname)s: %(message)s",
            "%Y-%m-%d %H:%M:%S",
        )
    )

    # add error file handler
    error_file_handler = logging.FileHandler(filename=error_log_filepath)
    error_file_handler.setLevel(logging.ERROR)
    error_file_handler.setFormatter(
        logging.Formatter(
            "%(asctime)s - %(name)s - %(levelname)s: %(message)s",
            "%Y-%m-%d %H:%M:%S",
        )
    )

    logger.addHandler(stream_handler)
    logger.addHandler(info_file_handler)
    logger.addHandler(error_file_handler)
    logger.propagate = False
    return logger

Overwriting /content/reighns/config/config.py


In [ ]:
%%writefile {BASE_DIR}/main.py
import logging
from config import config

def divide_by_zero(logger: logging.Logger):
    try:
        logger.info("I am trying to divide by zero!")
        1 / 0
    except ZeroDivisionError as e:
        logger.error(e)  # ERROR:root:division by zero
        logger.critical(e, exc_info=True)  # Logs error with stack trace

if __name__ == "__main__":
    train_logger = config.init_logger(
            logs_dir = config.LOGS_DIR,
            default_level = logging.DEBUG,
            stream_level=logging.INFO,
            module_name = "train")
    divide_by_zero(train_logger)

Overwriting /content/reighns/main.py


In [ ]:
!python main.py

2022-04-27 10:15:10 - train - INFO: I am trying to divide by zero!
2022-04-27 10:15:10 - train - ERROR: division by zero
2022-04-27 10:15:10 - train - CRITICAL: division by zero
Traceback (most recent call last):
  File "main.py", line 7, in divide_by_zero
    1 / 0
ZeroDivisionError: division by zero


## Workflow

### Workflow in IDE

### Workflow in Google Colab

In [ ]:
%%writefile {CONFIG_DIR}/config.py
import logging
import sys
from dataclasses import dataclass
from pathlib import Path
from typing import List, Optional

# Creating Directories
BASE_DIR = Path(__file__).parent.parent.absolute()

CONFIG_DIR = Path(BASE_DIR, "config")
LOGS_DIR = Path(BASE_DIR, "logs")
SRC_DIR = Path(BASE_DIR, "src")
DATA_DIR = Path(BASE_DIR, "data")
STORES_DIR = Path(BASE_DIR, "stores")
TEST_DIR = Path(BASE_DIR, "tests")

## Local stores
MODEL_REGISTRY = Path(STORES_DIR, "model")
RAW_DATA = Path(DATA_DIR, "raw")
PROCESSED_DATA = Path(DATA_DIR, "processed")

## Create dirs
for d in [
    CONFIG_DIR,
    LOGS_DIR,
    DATA_DIR,
    STORES_DIR,
    TEST_DIR,
    MODEL_REGISTRY,
    RAW_DATA,
    PROCESSED_DATA,
]:
    d.mkdir(parents=True, exist_ok=True)


# Logger
def init_logger(
    logs_dir: Path,
    default_level=logging.DEBUG,
    stream_level=logging.INFO,
    module_name: Optional[str] = None,
) -> logging.Logger:
    """Initialize logger.

    Args:
        logs_dir (Path): Path to log directory.
        default_level (int, optional): Default logging level. Defaults to logging.DEBUG.
        stream_level (int, optional): Stream logging level. Defaults to logging.INFO.
        module_name (Optional[str]): Module name to be used in logger. Defaults to None.

    Returns:
        logging.Logger: The logger object.

    Example:
        >>> import logging
        >>> import sys
        >>> from pathlib import Path
        >>> from typing import Optional
        >>> # Creating Directories
        >>> BASE_DIR = Path("__file__").parent.parent.absolute()
        >>> LOGS_DIR = Path(BASE_DIR, "logs")
        >>> LOGS_DIR.mkdir(parents=True, exist_ok=True)
        >>> train_logger = init_logger(LOGS_DIR, module_name="train")
        >>> # Logging levels (from lowest to highest priority)
        >>> try:
        >>>     train_logger.info("I am trying to divide by zero!")
        >>>     1 / 0
        >>> except ZeroDivisionError as e:
        >>>     train_logger.error(e)  # ERROR:root:division by zero
        >>>     train_logger.critical(e, exc_info=True)  # Logs error with stack trace
    """

    if module_name is None:
        logger = logging.getLogger(__name__)
        info_log_filepath = Path(logs_dir, "info.log")
        error_log_filepath = Path(logs_dir, "error.log")
    else:
        # get module name, useful for multi-module logging
        logger = logging.getLogger(module_name)
        info_log_filepath = Path(logs_dir, f"{module_name}_info.log")
        error_log_filepath = Path(logs_dir, f"{module_name}_error.log")

    logger.setLevel(default_level)
    stream_handler = logging.StreamHandler(stream=sys.stdout)
    stream_handler.setLevel(stream_level)
    stream_handler.setFormatter(
        logging.Formatter(
            "%(asctime)s - %(name)s - %(levelname)s: %(message)s",
            "%Y-%m-%d %H:%M:%S",
        )
    )

    info_file_handler = logging.FileHandler(filename=info_log_filepath)
    info_file_handler.setLevel(logging.INFO)
    info_file_handler.setFormatter(
        logging.Formatter(
            "%(asctime)s - %(name)s - %(levelname)s: %(message)s",
            "%Y-%m-%d %H:%M:%S",
        )
    )

    # add error file handler
    error_file_handler = logging.FileHandler(filename=error_log_filepath)
    error_file_handler.setLevel(logging.ERROR)
    error_file_handler.setFormatter(
        logging.Formatter(
            "%(asctime)s - %(name)s - %(levelname)s: %(message)s",
            "%Y-%m-%d %H:%M:%S",
        )
    )

    logger.addHandler(stream_handler)
    logger.addHandler(info_file_handler)
    logger.addHandler(error_file_handler)
    logger.propagate = False
    return logger

Overwriting /content/reighns/config/config.py


In [ ]:
%%writefile {BASE_DIR}/main.py
import logging
from config import config

def divide_by_zero(logger: logging.Logger):
    try:
        logger.info("I am trying to divide by zero!")
        1 / 0
    except ZeroDivisionError as e:
        logger.error(e)  # ERROR:root:division by zero
        logger.critical(e, exc_info=True)  # Logs error with stack trace

if __name__ == "__main__":
    train_logger = config.init_logger(
            logs_dir = config.LOGS_DIR,
            default_level = logging.DEBUG,
            stream_level=logging.INFO,
            module_name = "train")
    divide_by_zero(train_logger)

Overwriting /content/reighns/main.py


In [ ]:
!python main.py

2022-04-27 10:15:10 - train - INFO: I am trying to divide by zero!
2022-04-27 10:15:10 - train - ERROR: division by zero
2022-04-27 10:15:10 - train - CRITICAL: division by zero
Traceback (most recent call last):
  File "main.py", line 7, in divide_by_zero
    1 / 0
ZeroDivisionError: division by zero


> 

## TODO Log

- Each individual ML experiment should come with its own log file for clarity. That means, if we have a total of $3$ experiments of a ML project, named `exp_1, exp_2, exp_3`, then each of their log files should be separated accordingly as well.

- If we find ourself adding too many `handlers` to the function, then we may define a `logging` config like in [https://madewithml.com/courses/mlops/logging/](https://madewithml.com/courses/mlops/logging/).

- If can have one log file with multiple module references instead of multiple log files individually.

- Add timestamp prefix for logger (experiment).

- For more practices, one can refer to the references below.

## References

- https://docs.python.org/3/howto/logging-cookbook.html
- https://docs.python.org/3/library/logging.html#
- https://madewithml.com/courses/mlops/logging/
- Using logging in multiple modules: https://docs.python.org/3/howto/logging-cookbook.html